# 개요
[AI05] 경구약제 이미지 객체 검출(Object Detection) 프로젝트

이번 프로젝트의 목표는 사진 속에 있는 최대 4개의 알약의 이름(클래스)과 위치(바운딩 박스)를 검출하는 것

성능 지표: mAP@[0.75:0.95] 지표를 사용하여 모델의 성능을 측정

문제점: 데이터의 어노테이션이 없는 알약의 처리



In [ ]:
!pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.9 MB/s eta 0:00:00


In [ ]:
!pip install ultralytics

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math
import torchvision.transforms.functional as TF
import torch.nn.functional as F
import kagglehub
import torchvision
import pandas as pd
import torchvision.tv_tensors as tv_tensors


from tqdm import tqdm
from torch.utils.data import DataLoader, WeightedRandomSampler
from PIL import Image, ImageEnhance, ImageFilter
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, datasets
from torchvision.transforms import v2
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torchvision.models.detection.ssd import SSD300_VGG16_Weights

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

!apt-get update -qq
!apt-get install fonts-nanum* -qq

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings(action='ignore')

path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf' # 나눔 고딕
font_name = fm.FontProperties(fname=path, size=10).get_name() # 기본 폰트 사이즈 : 10
plt.rc('font', family=font_name)

fm.fontManager.addfont(path)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 39 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [ ]:
# device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.mps.is_available() else device)
print(f"Using device: {device}")

Using device: cuda


# 하이퍼 파라미터 튜닝

In [ ]:
batch_sizes = 16
epoch = 200

# Data Load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "/content/drive/MyDrive/codeit/project 1/team/data"

print("Path to dataset files:", path)


Path to dataset files: /content/drive/MyDrive/codeit/project 1/team/data


In [ ]:
train_dir = "/content/drive/MyDrive/codeit/project 1/team/data/images"
annotation_dir = "/content/drive/MyDrive/codeit/project 1/team/data/final_annotations.json"

###데이터 확인

In [ ]:
img_files = [f for f in os.listdir(train_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
print(f"Train 수: {len(img_files)}")



Train 수: 3176


#데이터 로더 및 데이터 셋

In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import os

class TestDataset(Dataset):
    def __init__(self, image_dir, image_list, transforms=None):
        self.image_dir = image_dir
        self.transforms = transforms
        self.image_files = image_list  # 테스트 이미지 리스트 (확장자 없음)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # 이미지 파일 경로
        image_file = self.image_files[idx]
        image_path = os.path.join(self.image_dir, image_file)

        # 이미지 로드
        image = Image.open(image_path).convert("RGB")

        # Transform 적용
        if self.transforms:
            image = self.transforms(image)

        return image, self.image_files[idx]  # 이미지와 파일 이름 반환


In [ ]:
import os
import json
import hashlib

def extract_classes_from_json_groups(json_groups):
    category_map = {}
    seen_hashes = set()

    for paths in json_groups.values():
        for path in paths:
            with open(path, 'r') as f:
                data = json.load(f)
                categories = data.get("categories", [])

                # 중복 방지: 같은 구조면 스킵
                hash_key = hashlib.md5(json.dumps(categories, sort_keys=True).encode()).hexdigest()
                if hash_key in seen_hashes:
                    continue
                seen_hashes.add(hash_key)

                for cat in categories:
                    category_map[cat["id"]] = cat["name"]


    sorted_categories = sorted(category_map.items())
    classes = ["__background__"] + [name for _, name in sorted_categories]

    return classes

In [ ]:
import json
from collections import defaultdict

# JSON 로드
with open(annotation_dir, "r", encoding="utf-8") as f:
    coco_data = json.load(f)

# ✅ 이미지별 어노테이션 매핑 생성
annotation_mapping = defaultdict(list)
for ann in coco_data["annotations"]:
    image_id = ann["image_id"]
    image_name = next(
        (img["file_name"] for img in coco_data["images"] if img["id"] == image_id),
        None
    )
    if image_name:
        annotation_mapping[image_name].append(ann)

print("✅ annotation_mapping 생성 완료")
print("예시 키:", list(annotation_mapping.keys())[:5])
print("첫 번째 이미지의 어노테이션 수:", len(annotation_mapping[list(annotation_mapping.keys())[0]]))


✅ annotation_mapping 생성 완료
예시 키: ['K-001900-016548-018110-021026_0_2_0_2_70_000_200.png', 'K-001900-016548-018110-021026_0_2_0_2_75_000_200.png', 'K-001900-016548-018110-027926_0_2_0_2_75_000_200.png', 'K-001900-016548-018110-029345_0_2_0_2_70_000_200.png', 'K-001900-016548-018110-029345_0_2_0_2_90_000_200.png']
첫 번째 이미지의 어노테이션 수: 4


In [ ]:
import json
with open(annotation_dir, "r", encoding="utf-8") as f:
    data = json.load(f)

cat_ids = sorted(set([ann["category_id"] for ann in data["annotations"]]))
print("카테고리 ID의 예시:", cat_ids[:50])
print("카테고리 개수:", len(cat_ids))


카테고리 ID의 예시: [1899, 2482, 3350, 3482, 3543, 3742, 3831, 4377, 4542, 5093, 5885, 6191, 6562, 10220, 12080, 12246, 12419, 12777, 13394, 13899, 16231, 16261, 16547, 16550, 16687, 18109, 18146, 18356, 19231, 19551, 19606, 19860, 20013, 20237, 20876, 21025, 21324, 21770, 22073, 22346, 22361, 22626, 23202, 23222, 24849, 25366, 25437, 25468, 27652, 27732]
카테고리 개수: 73


In [ ]:
from sklearn.model_selection import train_test_split

# 전체 이미지 파일 이름 리스트
image_names = list(annotation_mapping.keys())

# 이미지 리스트를 train/val로 나누기
train_list, val_list = train_test_split(image_names, test_size=0.2, random_state=42)

# 어노테이션 매핑도 함께 나누기
annotation_mapping_train = {name: annotation_mapping[name] for name in train_list}
annotation_mapping_val   = {name: annotation_mapping[name] for name in val_list}

print("✅ train/val 분리 완료")
print(f"Train 이미지 수: {len(train_list)}")
print(f"Val 이미지 수: {len(val_list)}")


✅ train/val 분리 완료
Train 이미지 수: 3052
Val 이미지 수: 763


In [ ]:
from torchvision.transforms import v2

train_transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize((640, 640)),
])

valid_transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize((640, 640)),
])

In [ ]:
import torchvision.tv_tensors as tv_tensors

class CustomDataset(Dataset):
    def __init__(self, image_dir, annotation_mapping, image_list, transforms=None):
        self.image_dir = image_dir
        self.annotation_mapping = annotation_mapping
        self.image_list = image_list
        self.transforms = transforms

    # ✅ 빠진 부분 추가!
    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        name = self.image_list[idx]
        image_path = os.path.join(self.image_dir, name)
        image = Image.open(image_path).convert("RGB")

        anns = self.annotation_mapping.get(name, [])
        boxes = [ann["bbox"] for ann in anns]
        labels = [ann["category_id"] for ann in anns]


        # ✅ BoundingBoxes 타입 변환 (v2용)
        image = tv_tensors.Image(image)
        boxes = tv_tensors.BoundingBoxes(
            boxes,
            format=tv_tensors.BoundingBoxFormat.XYXY,
            canvas_size=image.shape[-2:]
        )
        labels = torch.as_tensor(labels, dtype=torch.int64)

        # ✅ v2 transform 적용
        if self.transforms:
            image, boxes = self.transforms(image, boxes)

        # ✅ Normalize는 bbox 이후에만 적용
        image = self.normalize(image)

        target = {
            "boxes": boxes,
            "labels": labels,
            "image_id": idx
        }

        return image, target


In [ ]:
from torchvision.datasets import CocoDetection
from torchvision import transforms

train_dataset = CustomDataset(
    image_dir=train_dir,
    annotation_mapping=annotation_mapping_train,
    image_list=train_list,
    transforms=train_transform
)

valid_dataset = CustomDataset(
    image_dir=train_dir,  # validation 이미지도 동일 폴더 내에 있는 경우
    annotation_mapping=annotation_mapping_val,
    image_list=val_list,
    transforms=valid_transform
)


In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))


In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size = batch_sizes, shuffle=True, collate_fn=collate_fn)

valid_loader = DataLoader(valid_dataset, batch_size = batch_sizes, shuffle=False, collate_fn=collate_fn)


# 데이터 크기 출력
print(f"Train 데이터셋 크기: {len(train_dataset)}")
print(f"Validation 데이터셋 크기: {len(valid_dataset)}")

Train 데이터셋 크기: 3052
Validation 데이터셋 크기: 763


In [ ]:
print("Train 샘플 수:", len(train_dataset))
print("Val 샘플 수:", len(valid_dataset))
print("예시 train 이미지:", train_dataset.image_list[0])
print("예시 val 이미지:", valid_dataset.image_list[0])


Train 샘플 수: 3052
Val 샘플 수: 763
예시 train 이미지: synthetic_final_000200.png
예시 val 이미지: synthetic_final_000439.png


In [ ]:
# ============================================
# ✅ COCO → YOLO 변환 파트 (기존 데이터 로딩 이후 추가)
# ============================================

import os, json
from tqdm import tqdm

# 기존 path와 train_dir 그대로 사용
train_json = os.path.join(path, "merged_coco_final_train.json")
val_json   = os.path.join(path, "merged_coco_final_valid.json")

# ✅ (1) train/val JSON 파일 저장
# 기존 annotation_mapping_train / val 기반으로 저장
def save_coco_split_json(mapping, split_name, save_path):
    images = []
    annotations = []
    categories = coco_data["categories"]

    img_id_map = {}
    for i, name in enumerate(mapping.keys()):
        img_info = next((img for img in coco_data["images"] if img["file_name"] == name), None)
        if img_info:
            new_img = img_info.copy()
            new_img["id"] = i + 1
            img_id_map[img_info["id"]] = new_img["id"]
            images.append(new_img)

            for ann in mapping[name]:
                new_ann = ann.copy()
                new_ann["image_id"] = new_img["id"]
                annotations.append(new_ann)

    coco_split = {
        "images": images,
        "annotations": annotations,
        "categories": categories
    }

    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(coco_split, f, ensure_ascii=False, indent=2)
    print(f"✅ {split_name} JSON 저장 완료 → {save_path}")


save_coco_split_json(annotation_mapping_train, "train", train_json)
save_coco_split_json(annotation_mapping_val, "val", val_json)

# ✅ (2) YOLO 폴더 구조 생성
yolo_root = os.path.join(path, "yolo_dataset")
os.makedirs(os.path.join(yolo_root, "labels/train"), exist_ok=True)
os.makedirs(os.path.join(yolo_root, "labels/val"), exist_ok=True)
os.makedirs(os.path.join(yolo_root, "images/train"), exist_ok=True)
os.makedirs(os.path.join(yolo_root, "images/val"), exist_ok=True)


# ✅ (3) COCO → YOLO 변환 함수 (수정된 버전)
def coco_to_yolo(json_path, split):
    with open(json_path, "r", encoding="utf-8") as f:
        coco = json.load(f)

    # ✅ 카테고리 매핑 테이블 생성 (category_id → YOLO 인덱스)
    category_id_to_index = {cat["id"]: i for i, cat in enumerate(coco["categories"])}
    print(f"📦 category_id → class_index 매핑 예시: {list(category_id_to_index.items())[:5]}")

    images = {img["id"]: img for img in coco["images"]}
    ann_by_img = {img_id: [] for img_id in images.keys()}

    for ann in coco["annotations"]:
        ann_by_img[ann["image_id"]].append(ann)

    for img_id, anns in tqdm(ann_by_img.items(), desc=f"Converting {split}"):
        img_info = images[img_id]
        w, h = img_info["width"], img_info["height"]
        file_name = img_info["file_name"]
        file_stem = os.path.splitext(file_name)[0]
        label_path = os.path.join(yolo_root, f"labels/{split}/{file_stem}.txt")

        lines = []
        for ann in anns:
            # ✅ COCO: [x_min, y_min, width, height]
            x_min, y_min, box_w, box_h = ann["bbox"]

            x_center = (x_min + box_w / 2) / w
            y_center = (y_min + box_h / 2) / h
            box_w /= w
            box_h /= h

            # ✅ 잘못된 category_id 보정
            raw_id = ann["category_id"]
            cls_id = category_id_to_index.get(raw_id, 0)  # 존재하지 않으면 0으로 대체

            lines.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {box_w:.6f} {box_h:.6f}")

        with open(label_path, "w", encoding="utf-8") as lf:
            lf.write("\n".join(lines))

    print(f"✅ {split} 변환 완료 → {len(images)}개 이미지")


# ✅ (4) 변환 실행
coco_to_yolo(train_json, "train")
coco_to_yolo(val_json, "val")


# ✅ (5) data.yaml 생성
yaml_path = os.path.join(yolo_root, "data.yaml")
categories = json.load(open(train_json))["categories"]

with open(yaml_path, "w", encoding="utf-8") as f:
    f.write(f"train: {yolo_root}/images/train\n")
    f.write(f"val: {yolo_root}/images/val\n")
    f.write(f"nc: {len(categories)}\n")
    f.write("names:\n")
    for i, c in enumerate(categories):
        f.write(f"  {i}: {c['name']}\n")

print(f"🎯 data.yaml 생성 완료 → {yaml_path}")


✅ train JSON 저장 완료 → /content/drive/MyDrive/codeit/project 1/team/data/merged_coco_final_train.json
✅ val JSON 저장 완료 → /content/drive/MyDrive/codeit/project 1/team/data/merged_coco_final_valid.json
📦 category_id → class_index 매핑 예시: [(16547, 0), (32309, 1), (33879, 2), (3482, 3), (31704, 4)]


Converting train: 100%|██████████| 3052/3052 [00:16<00:00, 182.92it/s]


✅ train 변환 완료 → 3052개 이미지
📦 category_id → class_index 매핑 예시: [(16547, 0), (32309, 1), (33879, 2), (3482, 3), (31704, 4)]


Converting val: 100%|██████████| 763/763 [02:40<00:00,  4.75it/s]


✅ val 변환 완료 → 763개 이미지
🎯 data.yaml 생성 완료 → /content/drive/MyDrive/codeit/project 1/team/data/yolo_dataset/data.yaml


In [ ]:
# ================================================
# 🧩 여러 개의 COCO 구조 JSON 병합기 (자동 카테고리 매핑)
# ================================================
import os, json
from tqdm import tqdm

base_dir = "/content/drive/MyDrive/codeit/project 1/data"
anno_root = os.path.join(base_dir, "train_annotations")

json_files = []
for dirpath, _, filenames in os.walk(anno_root):
    for file in filenames:
        if file.endswith(".json"):
            # ⚠️ final_annotations 계열 파일은 제외 (중간 산출물)
            if file.lower().startswith("final_annotations"):
                continue
            json_files.append(os.path.join(dirpath, file))

print(f"📦 병합 대상 JSON 파일: {len(json_files)}개")

# COCO 구조 초기화
merged = {"images": [], "annotations": [], "categories": []}
img_id, ann_id = 1, 1

# 카테고리 매핑 (category_id → global id)
global_cat_map = {}
next_cat_id = 1

for jf in tqdm(json_files, desc="Merging COCO JSONs"):
    try:
        with open(jf, "r", encoding="utf-8") as f:
            data = json.load(f)
    except Exception as e:
        print(f"⚠️ {jf} 로드 실패: {e}")
        continue

    # ✅ 카테고리 매핑
    for cat in data.get("categories", []):
        cat_name = cat.get("name")
        if not cat_name:
            continue
        if cat_name not in global_cat_map:
            global_cat_map[cat_name] = next_cat_id
            next_cat_id += 1

    # ✅ 이미지
    for img in data.get("images", []):
        new_img = img.copy()
        new_img["id"] = img_id
        merged["images"].append(new_img)
        image_id = img_id
        img_id += 1

    # ✅ 어노테이션
    for ann in data.get("annotations", []):
        new_ann = ann.copy()
        # category_id 재매핑
        cat_name = None
        if "category_id" in ann and len(data.get("categories", [])) > 0:
            for c in data["categories"]:
                if c["id"] == ann["category_id"]:
                    cat_name = c["name"]
                    break
        if cat_name and cat_name in global_cat_map:
            new_ann["category_id"] = global_cat_map[cat_name]
        else:
            continue

        new_ann["id"] = ann_id
        new_ann["image_id"] = image_id
        merged["annotations"].append(new_ann)
        ann_id += 1

# ✅ 카테고리 정리
merged["categories"] = [{"id": v, "name": k} for k, v in sorted(global_cat_map.items(), key=lambda x: x[1])]

# ✅ 저장
output_path = os.path.join(anno_root, "final_annotations_coco_merged.json")
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(merged, f, ensure_ascii=False, indent=2)

print(f"\n✅ 병합 완료 → {output_path}")
print(f"📊 이미지 수: {len(merged['images'])}")
print(f"📊 어노테이션 수: {len(merged['annotations'])}")
print(f"📊 카테고리 수: {len(merged['categories'])}")
print("📋 카테고리 예시:", [c["name"] for c in merged["categories"][:10]])


📦 병합 대상 JSON 파일: 4526개


Merging COCO JSONs: 100%|██████████| 4526/4526 [00:07<00:00, 579.09it/s]



✅ 병합 완료 → /content/drive/MyDrive/codeit/project 1/data/train_annotations/final_annotations_coco_merged.json
📊 이미지 수: 4526
📊 어노테이션 수: 4526
📊 카테고리 수: 73
📋 카테고리 예시: ['가바토파정 100mg', '란스톤엘에프디티정 30mg', '울트라셋이알서방정', '보령부스파정 5mg', '신바로정', '동아가바펜틴정 800mg', '낙소졸정 500/20mg', '펠루비정(펠루비프로펜)', '비모보정 500/20mg', '레일라정']


In [ ]:
# ============================================
# ✅ COCO → YOLO 변환 + data.yaml 생성
# ============================================
import os, json
from tqdm import tqdm

base_dir = "/content/drive/MyDrive/codeit/project 1/data"
coco_path = os.path.join(base_dir, "train_annotations", "final_annotations_coco_merged.json")

# 변환 폴더 구조 생성
yolo_root = os.path.join(base_dir, "yolo_dataset")
os.makedirs(os.path.join(yolo_root, "labels/train"), exist_ok=True)
os.makedirs(os.path.join(yolo_root, "images/train"), exist_ok=True)

# COCO 로드
with open(coco_path, "r", encoding="utf-8") as f:
    coco = json.load(f)

images = {img["id"]: img for img in coco["images"]}
annotations = coco["annotations"]
categories = coco["categories"]

# category_id → YOLO class index 매핑
cat_map = {cat["id"]: i for i, cat in enumerate(categories)}

# 변환
for ann in tqdm(annotations, desc="Converting COCO → YOLO"):
    img_info = images[ann["image_id"]]
    w, h = img_info["width"], img_info["height"]
    file_name = img_info["file_name"]
    file_stem = os.path.splitext(file_name)[0]

    # bbox → YOLO 형식
    x_min, y_min, box_w, box_h = ann["bbox"]
    x_center = (x_min + box_w / 2) / w
    y_center = (y_min + box_h / 2) / h
    box_w /= w
    box_h /= h
    cls = cat_map[ann["category_id"]]

    label_path = os.path.join(yolo_root, "labels/train", f"{file_stem}.txt")
    with open(label_path, "a", encoding="utf-8") as f:
        f.write(f"{cls} {x_center:.6f} {y_center:.6f} {box_w:.6f} {box_h:.6f}\n")

print("✅ YOLO 변환 완료!")

# data.yaml 생성
yaml_path = os.path.join(yolo_root, "data.yaml")
with open(yaml_path, "w", encoding="utf-8") as f:
    f.write(f"train: {yolo_root}/images/train\n")
    f.write(f"val: {yolo_root}/images/train\n")  # 아직 val 안 나눴으니 train 재사용
    f.write(f"nc: {len(categories)}\n")
    f.write("names:\n")
    for i, c in enumerate(categories):
        f.write(f"  {i}: {c['name']}\n")

print(f"🎯 data.yaml 생성 완료 → {yaml_path}")


Converting COCO → YOLO: 100%|██████████| 4526/4526 [00:16<00:00, 275.02it/s]

✅ YOLO 변환 완료!
🎯 data.yaml 생성 완료 → /content/drive/MyDrive/codeit/project 1/data/yolo_dataset/data.yaml


In [ ]:
from ultralytics import YOLO
# 🚀 YOLO 모델 로드 및 학습
model = YOLO("yolo11m.pt")
results = model.train(
    data="/content/drive/MyDrive/codeit/project 1/team/data/yolo_dataset/data.yaml",
    epochs = epoch,
    imgsz=640,
    verbose=False,   # 세부 로그 제거
    plots=False,     # 라벨 시각화 안 함
    exist_ok=True
)


Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/codeit/project 1/team/data/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patienc

In [ ]:
from google.colab import files

# best.pt 다운로드
files.download("/content/runs/detect/train/weights/best.pt")

# 또는 last.pt 다운로드
# files.download("/content/runs/detect/train/weights/last.pt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 2차 재학습

In [ ]:
from ultralytics import YOLO

# 🔹 이전 학습의 최고 성능 가중치 경로
# 예: /content/runs/detect/train/weights/best.pt
prev_best = "/content/runs/detect/train/weights/best.pt"

# 🔹 모델 로드 (이전 best.pt)
model = YOLO(prev_best)

# 🔹 재학습 시작 (새로운 데이터나 같은 데이터에서 추가 fine-tuning)
results = model.train(
    data="/content/drive/MyDrive/codeit/project 1/data/yolo_dataset/data.yaml",  # 새로 만든 YAML 경로
    epochs=epoch,             # 추가 학습 epoch
    imgsz=640,             # 이미지 크기
    batch=16,              # 배치 크기
    device=0,              # GPU 자동 선택
    optimizer="AdamW",     # 작은 객체에 안정적
    lr0=0.0005,            # 기존보다 살짝 낮은 학습률 권장
    cos_lr=True,           # Cosine 스케줄러
    warmup_epochs=2,       # 빠른 안정화
    patience=20,           # Early stopping
    name="retrain_best",   # 새로운 실험 이름
    exist_ok=True,         # 동일 이름 덮어쓰기 허용
    verbose=True,          # 학습 로그 출력
    plots=True,             # 시각화 그래프 저장
    overlap_mask=False
)

print("\n✅ Fine-tuning completed successfully!")
print(f"📁 Results saved to: {model.trainer.save_dir}")


Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/codeit/project 1/data/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/runs/detect/train/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=retrain_best, nbs=64, nms=False, opset=None, optimize=False, optimiz